In [40]:
import pandas as pd
import numpy as np

In [88]:
cities = pd.read_csv('WDataFiles/WCities.csv')
gameCities = pd.read_csv('WDataFiles/WGameCities.csv')
tourneyResults = pd.read_csv('WDataFiles/WNCAATourneyCompactResults.csv')
seeds = pd.read_csv('WDataFiles/WNCAATourneySeeds.csv')
slots = pd.read_csv('WDataFiles/WNCAATourneySlots.csv')
seasonResults = pd.read_csv('WDataFiles/WRegularSeasonCompactResults.csv')
seasons = pd.read_csv('WDataFiles/WSeasons.csv')
teams = pd.read_csv('WDataFiles/WTeams.csv')
spellings = pd.read_csv('WDataFiles/WTeamSpellings.csv', encoding='latin1')

### Head

In [16]:
cities.head(2)

,CityID,City,State
0,4001,Abilene,TX
1,4002,Akron,OH


In [17]:
gameCities.head(2)

,Season,DayNum,WTeamID,LTeamID,CRType,CityID
0,2015,11,3103,3127,Regular,4002
1,2015,11,3104,3197,Regular,4353


In [18]:
tourneyResults.head(2)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,137,3104,94,3422,46,H,0
1,1998,137,3112,75,3365,63,H,0


In [47]:
seeds.head(2)

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163


In [20]:
slots.head(2)

,Slot,StrongSeed,WeakSeed
0,R1W1,W01,W16
1,R1W2,W02,W15


In [23]:
seasonResults.head(2)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1998,18,3104,91,3202,41,H,0
1,1998,18,3163,87,3221,76,H,0


In [27]:
seasons.head(2)

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1998,10/27/1997,East,Midwest,Mideast,West
1,1999,10/26/1998,East,Mideast,Midwest,West


In [30]:
teams.head(2)

,TeamID,TeamName
0,3101,Abilene Chr
1,3102,Air Force


In [89]:
spellings.head(2)

,TeamNameSpelling,TeamID
0,a&m-corpus chris,3394
1,abilene chr,3101


## Build Team Table

In [68]:
def rs_win_percent(df):
    '''
    Determine percent of wins in regular season
    '''
    for team, year in df.index:
        season = seasonResults[ seasonResults.Season == year ]
        wins = len(season[ season.WTeamID == team ])
        losses = len(season[ season.LTeamID == team ])
        win_perc = wins / (wins + losses)
        df.set_value((team, year), 'rs_win_perc', win_perc)

In [81]:
def last_games_percent(df, games=10):
    '''
    Determine percent of wins in last x games of regular season
    '''
    for team, year in df.index:
        season = seasonResults[ (seasonResults.Season == year) & ((seasonResults.WTeamID == team) | (seasonResults.LTeamID == team)) ].sort_values('DayNum', ascending=False).head(games)
        wins = len(season[ season.WTeamID == team ])
        losses = len(season[ season.LTeamID == team ])
        win_perc = wins / (wins + losses)
        df.set_value((team, year), 'last_{}_perc'.format(games), win_perc)

In [91]:
def seed_ranking(row, df):
    '''
    Create rough ranking from seed (treat all regions equally)
    '''
    seed_loc = df.columns.get_loc("Seed")
    seed = row[seed_loc]
    return int(seed[1:]) * 4

In [92]:
tt = seeds.set_index(['TeamID', 'Season']).join(teams.set_index('TeamID'))
rs_win_percent(tt)
last_games_percent(tt, 5)
last_games_percent(tt, 10)
last_games_percent(tt, 15)
tt['seed_ranking'] = tt.apply(seed_ranking, df=tt, axis=1)
tt.head()

/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.


,,Seed,TeamName,rs_win_perc,last_5_perc,last_10_perc,last_15_perc,seed_ranking
TeamID,Season,,,,,,,
3330,1998,W01,Old Dominion,0.925926,1.0,0.9,0.866667,4
3163,1998,W02,Connecticut,0.937500,1.0,0.9,0.933333,8
3112,1998,W03,Arizona,0.777778,0.8,0.8,0.800000,12
3301,1998,W04,NC State,0.777778,0.6,0.6,0.666667,16
3272,1998,W05,Memphis,0.846154,1.0,0.9,0.933333,20


/home/luke/anaconda2/envs/py3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """


In [90]:
tt

KeyError: 1

### Model
Plan:
- For each matchup:
    - Take home team numbers and subtract away team
    - Consider 1 a win and 0 a loss